In [1]:
from transformers import BertForMaskedLM, BertConfig,BertTokenizer, BertForPreTraining, BertForSequenceClassification, AutoModel
import torch
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader, SequentialSampler
from tqdm import tqdm
from typing import List,Union,Dict
import torch.nn.functional as F
import numpy as np
import transformers
from torch import Tensor

In [2]:
from input_marg_1 import calculate_input_marginalisation

In [3]:
dataset = load_dataset('glue', 'sst2', split = 'validation')

Reusing dataset glue (C:\Users\iamab\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [4]:
dataset.features['label']

ClassLabel(num_classes=2, names=['negative', 'positive'], names_file=None, id=None)

In [5]:
def give_sent_label_pair(index):
    print(dataset[index]['sentence'])
    print(dataset[index]['label'])

    return dataset[index]['sentence'],dataset[index]['label']

In [6]:
sent,label = give_sent_label_pair(1)

unflinchingly bleak and desperate 
0


In [7]:
target_model = BertForSequenceClassification.from_pretrained('textattack/bert-base-uncased-SST-2')

In [8]:
language_model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [10]:
ats,md = calculate_input_marginalisation(target_model, language_model, sent, tokenizer, label)

calculating target class probs for index 9: 100%|████████████████████████████████████████████████████████████████| 342/342 [00:08<00:00, 38.74it/s]


In [11]:
ats

{'[CLS]': -0.1743878299905326,
 'unflinchingly': 4.321102052523051,
 'bleak': 7.044154536846676,
 'and': 1.3517100341011483,
 'desperate': 0.2528024051361477,
 '[SEP]': 8.374740626325515}

In [35]:
list(md.items())

[('[CLS]', 0.9872812628746033),
 ('un', 1e-08),
 ('##fl', 0.984134316444397),
 ('##in', 0.9893656969070435),
 ('##ching', 0.9960137605667114),
 ('##ly', 0.9046838879585266),
 ('bleak', 0.34262144565582275),
 ('and', 0.9642247557640076),
 ('desperate', 0.9829728603363037),
 ('[SEP]', 0.17165744304656982)]

In [41]:
dic = {}
items= list(md.items())


flag = True
ind= list(range(len(items)))

for i in ind:
    word = items[i][0]
    prob = items[i][1]
    k = 0
    for j in items[i+1:]:
        if '##' in j[0]:
            word += j[0].strip('#')
            prob += j[1]
            k+=1
            ind.remove(i+k)
        else:
            break
    dic[word] = prob/(k+1)
#     i += (k+1)

In [39]:
(md['un'] + md['##fl'] + md ['##in'] + md['##ching'] + md['##ly'])/(5)

0.7748395343753357

In [42]:
dic

{'[CLS]': 0.9872812628746033,
 'unflinchingly': 0.7748395343753357,
 'bleak': 0.34262144565582275,
 'and': 0.9642247557640076,
 'desperate': 0.9829728603363037,
 '[SEP]': 0.17165744304656982}

In [10]:
tokens = tokenizer.convert_ids_to_tokens(tokenizer(sent)['input_ids'])

In [11]:
for a,b in zip(tokens, att_scores.values()):
    print(a,b)

[CLS] 0.4772483666697922
pumpkin 0.029969691004827048
takes 0.30724200160807813
an -0.4556449524076047
ad 30.600218085520094
##mir -2.3149723971849347
##able -0.3817417560742715
look -0.9226856943233601
at 0.018292548050501622
the -0.13783306691372754
h -0.12477640857941186
##yp 30.600218085520094
##oc -0.24116085970290335
##ris -0.31669134652846775
##y 0.19031770140023285
of -0.09110892242724766
political -0.009992301079638999
correct 0.29630282761373516
##ness -0.07191547941736953
, 0.7221702795384521
but 6.879198969639418
it 1.5363433169654614
does 0.43830414754857383
so 3.1318572413116836
with 0.08312772822510661
such 0.6798312113387914
an 0.8711515549013571
uneven 8.389592975286496
tone 2.884565146915489
that -0.542087147248183
you -1.050486450889875
never -0.0056901795486687945
know 3.209775622044061
when -0.1972031858122305
humor 1.4144783063796802
ends 0.5421301989021878
and -0.17224572454760523
tragedy -0.7424034604628043
begins 0.12855522687144694
. -0.24872171140183585
[SEP]

In [21]:
a.repeat(3,1)

tensor([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]])

IndexError: too many indices for tensor of dimension 1

In [29]:
x = torch.cat(x, dim =-1)

In [33]:
y = torch.arange(12)

In [35]:
x@y

tensor(572)

In [42]:
x.ndim

1

In [19]:
from datasets import Dataset

In [22]:
dl = DataLoader(dataset, batch_size= 4, sampler = SequentialSampler(dataset))

In [36]:
dl.batch_size

4

In [7]:
class Ds():
def  __init__(self, a,b,c):
self.a  = a
self.b = b
self.c = c

def __getitem__(self, i): return {'a':self.a[i], 'b':self.b[i], 'c':self.c[i]}

In [8]:
a = torch.arange(20)
b = torch.arange(20,40)
c=  torch.arange(40,60)

In [9]:
ds = Ds(a,b,c)

In [10]:
ds[:4]

{'a': tensor([0, 1, 2, 3]),
 'b': tensor([20, 21, 22, 23]),
 'c': tensor([40, 41, 42, 43])}

In [18]:
dataset = prepare_dataset('sst2', tokenizer = tokenizer, split = ['validation'])

Reusing dataset glue (C:\Users\iamab\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [21]:
sample = dataset['validation'][:4]

In [15]:
import numpy as np

In [32]:
a = torch.tensor([1,2,3,4])
b = [10,11,12,13,14,15]
ls = []

In [33]:
for i in range(len(b)):
a_clone = a.clone()
a_clone[1] = b[i]  
ls.append(a_clone)

[tensor([ 1, 10,  3,  4]),
 tensor([ 1, 11,  3,  4]),
 tensor([ 1, 12,  3,  4]),
 tensor([ 1, 13,  3,  4]),
 tensor([ 1, 14,  3,  4]),
 tensor([ 1, 15,  3,  4])]

In [27]:
a

tensor([1, 2, 3])

In [28]:
b = a.clone()

In [30]:
b[2] = 500

In [1]:
from transformers import BertTokenizer

In [12]:
tok = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
inp = tok('Movie was brilliant and awesome lol', return_tensors = 'pt')

In [19]:
inp['input_ids'][0]

tensor([  101,  3185,  2001,  8235,  1998, 12476,  8840,  2140,   102])

In [20]:
tok.convert_ids_to_tokens(inp['input_ids'][0])

['[CLS]', 'movie', 'was', 'brilliant', 'and', 'awesome', 'lo', '##l', '[SEP]']